<table>
  <tr>
    <td><p style="font-size:45px; color: #55BBD2">Analysis of light microscopy images in Python </p></td>
    <td><img src="../ressources/lmb_logo.svg" alt="LMB Logo" width="500" height="600" align="right"></td>
  </tr>
</table>
<table>
  <tr>
    <td><p style="font-size:15px; color: #55BBD2">Version: September 2025</p></td>
  </tr>
</table>

# Part 1: Image and its visualisation

The course aims to introduce you to the essential knowledge to image processing and analysis of microscopic data in Python. 

In this part 1 of the course, we hope the course attendees will be able to:
- run a Python notebook,
- load and read microscopic images and metadata using the package bioio,
- understand what an image is,
- visualise different channel or section of an image using the matplotlib library,
- visualise different channel or section of an image using the napari plugin.

### Definitions:
* A module is defined as a set of related code or functions saved in a file with the extension .py. 
* A library is a general programming term to describe a collection of modules. 
* A Python package is a library. 
* A notebook is a file with the extension .ipynb.

### List of packages and modules to use in this section:
* <i>pathlib</i> : a module representing filesystem paths.
* <i>bioio</i> : a package for Image Reading, Metadata Conversion, and Image Writing for Microscopy Images in Pure Python.
* <i>numpy</i> : a Python library used for working with arrays.
* <i>matplotlib</i> : a comprehensive library for creating static, animated, and interactive visualizations in Python.
* <i>napari</i> : a fast, interactive, multi-dimensional image viewer for Python. It can be installed as a Python package. 

## Load and read data and metadata using bioio


In [ ]:
from pathlib import Path

data_folder = Path('../data')

In [ ]:
from bioio import BioImage

image = BioImage(Path(data_folder, 'airyscan-4colors.tif'))

The image and its metadata are now loaded into the variable "image". Next, we read the image and the metadata. 

### Metadata

Metadata stores information describing the data. It includes the number of channels in the images, the name of each channel and emission wavelength associated with each channel, the voxel size, and many other acquisition parameters.

In [ ]:
image.data
image.metadata
image.physical_pixel_sizes
image.shape
image.dims
image.channel_names

<div class="alert alert-success">

#### Exercise 

In the next cell, get familiar with the contents of the variable "image" and the metadata by printing the values stored in it one by one and understand the outputs. 
    
Hint: use the function print()
    
</div>

### Image dimension and pixel sizes

Next, we will check the different dimensions of our image. To illustrate this, we need to know what a dictionnary is as it is required here.

In [ ]:
image.dims

In [ ]:
for dim, element in enumerate(image.dims.order):
    print(f'Array axis {dim} is {element} with size {image.dims[element][0]}')

print(f'The name of the channels are {image.channel_names}')
print(f'The pixel size in X is {image.physical_pixel_sizes}')
print(f'The pixel size in X is {image.physical_pixel_sizes.X:.4f} microns')
print(f'The pixel size in Y is {image.physical_pixel_sizes.Y:.4f} microns')

### What is an image?

A digital image is a finite numeric representation of the intensity values. It is composed of picture elements known as pixels. 

In Python, it can be manipulated as a N-dimensional array using the class numpy.ndarray.

In [ ]:
# import the numpy package used for manipulating images
import numpy as np

# get the array of pixels intensity as a numpy object
data = image.data

# Print the type of the array
print('The data is a ', type(data)) 

# Print the dimension of the array
print('The array has the following shape', data.shape) 

# Print the physical pixel sizes and the types
pixel = image.physical_pixel_sizes
print('The physical pixel sizes are', pixel)
print('The pixel type is a', type(pixel)) 


## Visualize with matplotlib
### Display an image

In [ ]:
# import matplotlib for loading images
import matplotlib.pyplot as plt

# Let's display the second channel as a 2D image in a figure
plt.imshow(data[0,1,0,:,:], cmap='gray')

<div class="alert alert-success">
       
#### Exercise 

Display the second and third channels in the image and change the colormap cmap value to 'hot'.
    
Hint: use the same function that in the previous cell.
</div>

Now let's display all the channels of the image in a subplot using a `for` loop:

In [ ]:
# import matplotlib for displaying images
import matplotlib.pyplot as plt

# compute the number of channels
num_channels = image.shape[1]

# Display the image for each channel
fig, ax = plt.subplots(1, num_channels, figsize=(16,4))
for k  in range(num_channels):    
    ax[k].imshow(image.data[0,k,0], cmap='hot')
    ax[k].set_axis_off()
    ax[k].set_title(image.channel_names[k])    

In the following cell, we crop a part of the image, display it in a figure and overlay the pixel values.

In [ ]:
# Crop and downsample the array by a factor of 5
crop = image.data[0, 0, 0, 850:950:5, 900:1000:5] 

# Create a figure with axes
fig, ax = plt.subplots(figsize=(5, 5))

# Display the downsampled array in the figure
plt.imshow(crop, cmap='gray')

# Add the values of the pixel intensity
for i in range(crop.shape[0]):
    for j in range(crop.shape[1]):
        c = 'white' if crop[i, j] < 5 else 'black'
        ax.text(j, i, str(int(crop[i, j])), color=c, ha='center', va='center', size=10)
ax.axis("off")
plt.title('Pixel values');

<div class="alert alert-success">
       
#### Exercise 
    
Modify the previous cell to

1. select a different region within the image and display an overlay of the image with the pixel values,
2. check again the size of the image to be sure of not going out of bounds,
3. choose an appropriate downsampling factor,
4. adjust the figsize for a proper and visible display if necessary.
    
</div>

### Plot with matplotlib

Let say we would like to visualize the profile of the intensity distribution of a given channel along the x-axis at the mid-position of the y-axis in the image. 

In [ ]:
c = 3                       # channel number
y = data[0, 3, 0, 956, :]   # select the intensity to display
x = np.arange(0, len(y))    # set the x-range

plt.plot(x, y)

plt.xlabel("x [pixels]")
plt.ylabel("Intensity [a.u]")
plt.title("Intensity for channel " + str(c))

## Display with napari viewer

Napari is a fast, interactive, and open-source Python tool for viewing, annotating, and analyzing large multi-dimensional scientific images. It allows users to visualize 2D, 3D, and higher-dimensional data on a single canvas, overlay derived data like points and segmentations, and seamlessly integrate exploration with computation and annotation.

In [ ]:
import napari

viewer = napari.Viewer()
viewer.add_image(data)

Similar to Fiji, we can also hide or show some channels using what is called `layer` in the viewer. The layer can have properties such as `name`, `opacity`, `blending`, etc. 

<div class="alert alert-success">
       
#### Exercise 

To split the channels into layers, we need to specify the channel axis.

1. Add `channel_axis=1` into the add_image function in the previous cell and observe the difference from the previous viewer. 
2. Select a channel and play with the gamma or/and contrast limits parameters to adjust the contrast of each of the channel. 
3 The blending is by default additive. Select Channel 2 (dots) and set its blending to opaque. Hide and show this channel and observe the difference.
    
</div>

Now we will try to code the actions that one can manually do on the viewer.

<div class="alert alert-success">
       
#### Exercise 
    
Use the same viewer as before or create a new one. We will display the name of the channels as set in the metadata and set the contrast limits as the minimal and maximal intensity values.

1. set the `name` property to be the image.channel_names or an user-defined list of channels names,
2. create a list of min and max intensity value for the 4 channels,
3. set the `contrast_limits` property of each channel to range between the min and max intensity by attributing the `contrast_limits` variable to the created list in step 2,

Hint: use np.min(data[:, n, :, :, :]) and np.max(data[:, n, :, :, :]) to find the min and max of the n-th channel respectively
    
</div>

In [ ]:
import numpy as np

contrast_limits = [[np.min(data[:, n, :, :, :]), np.max(data[:, n, :, :, :])] for n in np.arange(data.shape[1])]
contrast_limits

### Add points layer 

Points are another types of layers that one can add to a napari viewer. It requires the coordinates of the points, (x, y) for 2D data points and (x, y, z) for 3D data points.

In [ ]:
points = np.array([[100, 100], [200, 200], [300, 100]]) # (y, x)
viewer.add_points(points, size=30, face_color="red", text=["pggkkk1", "p2", "p3"])

### Add labels layer

A label layer is an image-like layer where every pixel contains an integer ID corresponding to the region it belongs to. 

To illustrate this, we will create a labels data of the same size as the 2D image channel.  

In [ ]:
shape = data.shape

# Example data for labels layer
labels_data = np.zeros(shape[3::], dtype=int)
labels_data[10:300, 10:300] = 1
labels_data[500:900, 500:600] = 2

viewer.add_labels(labels_data)

We can also group the properties we want to attribute to a layer in a dictionnary and add it into the display funtion. 

Here we introduce the `colormap` of the labels and visibility `visible` parameters to the list of label parameters.

For a label layer, the colormap needs to be in a dictionary format such that the region IDs are the keys and the RGB triplet code colors are the values.

In [ ]:
# specify the colors of the labels in a dictionary
cols = {0: [0,0,0], 1: [1,0,0], 2:[0,1,0]}

# Properties dictionary for labels
labels_properties = {
    "name": "My Labels",
    "opacity": 0.5,
    "colormap": cols,           # color map for labels
    "visible": True
}

# Add to viewer
viewer.add_labels(labels_data, **labels_properties)